In [9]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from scipy.misc import imread
from top_level_features import hog_features
from top_level_features import color_histogram_hsv
from top_level_features import extract_features
import cPickle as pickle
import numpy as np
import os

In [16]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

def load_CIFAR_one(filename):
    with open(filename, 'rb') as f:
        datadict = pickle.load(f)
        X = datadict['data']
        Y = datadict['labels']
        return X, np.array(Y)

def load_CIFAR10(PATH):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_one(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    trSet = np.concatenate((Xtr, Ytr.reshape(50000,1)), 1)
    np.random.shuffle(trSet)
    Xval = trSet[:10000,:3072]
    Yval = trSet[:10000,3072]
    del X, Y
    Xte, Yte = load_CIFAR_one(os.path.join(PATH, 'test_batch'))
    return Xtr, Ytr, Xte, Yte, Xval, Yval

label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog',\
'frog', 'horse', 'ship', 'truck']

In [17]:
Xtr, Ytr, Xte, Yte, Xval, Yval = load_CIFAR10(".")

In [11]:
model = DecisionTreeClassifier()
param_dist = range(1,11)
random_search = GridSearchCV(estimator=model, param_grid=dict(max_depth=param_dist))
random_search.fit(Xtr,Ytr)
report(random_search.cv_results_)

Model with rank: 1
Mean validation score: 0.292 (std: 0.002)
Parameters: {'max_depth': 9}

Model with rank: 2
Mean validation score: 0.290 (std: 0.001)
Parameters: {'max_depth': 10}

Model with rank: 3
Mean validation score: 0.290 (std: 0.001)
Parameters: {'max_depth': 8}



In [15]:
featuresHog = extract_features(Xtr,[hog_features]) #extrae hog features
featuresHogTe = extract_features(Xte,[hog_features]) #extrae hog features
featuresHogVal = extract_features(Xval,[hog_features]) #extrae hog features
Ytr = to_categorical(Ytr)
Yte = to_categorical(Yte)
Yval = to_categorical(Yval)

random_search2 = GridSearchCV(estimator=model, param_grid=dict(max_depth=param_dist))
random_search2.fit(featuresHog,Ytr)
report(random_search.cv_results_)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
Model with rank: 1
Mean validation score: 0.292 (std: 0.002)
Parameters: {'max_depth': 9}

Model with rank: 2
Mean validation score: 0.290 (std: 0.001)
Parameters: {'max_depth': 10}

Model with rank: 3
Mean validation score: 0.290 (std: 0.001)
Parameters: {'max_depth': 8}



In [19]:
featuresHist = extract_features(Xtr,[color_histogram_hsv]) #extrae hog features
featuresHistTe = extract_features(Xte,[color_histogram_hsv]) #extrae hog features
featuresHistVal = extract_features(Xval,[color_histogram_hsv]) #extrae hog features

random_search3 = GridSearchCV(estimator=model, param_grid=dict(max_depth=param_dist))
random_search3.fit(featuresHist,Ytr)
report(random_search.cv_results_)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
Model with rank: 1
Mean validation score: 0.292 (std: 0.002)
Parameters: {'max_depth': 9}

Model with rank: 2
Mean validation score: 0.290 (std: 0.001)
Parameters: {'max_depth': 10}

Model with rank: 3
Mean validation score: 0.290 (std: 0.001)
Parameters: {'max_depth': 8}



In [20]:
features = extract_features(Xtr,[hog_features, color_histogram_hsv]) #extrae hog features
featuresTe = extract_features(Xte,[hog_features, color_histogram_hsv]) #extrae hog features
featuresVal = extract_features(Xval,[hog_features, color_histogram_hsv]) #extrae hog features

random_search4 = GridSearchCV(estimator=model, param_grid=dict(max_depth=param_dist))
random_search3.fit(features,Ytr)
report(random_search.cv_results_)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
Model with rank: 1
Mean validation score: 0.292 (std: 0.002)
Parameters: {'max_depth': 9}

Model with rank: 2
Mean validation score: 0.290 (std: 0.001)
Parameters: {'max_depth': 10}

Model with rank: 3
Mean validation score: 0.290 (std: 0.001)
Parameters: {'max_depth': 8}

